In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import tree
import random
import collections
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
resp = pd.read_csv("young-people-survey/responses.csv")
podcasts=pd.read_csv("young-people-survey/df_popular_podcasts.csv")
word_exp=re.compile("[^\x00-\x7F]+")
non_eng_pod=[index for index,value in enumerate(list(podcasts["Name"].to_dict().values())) if len(re.findall(word_exp,value))!=0]
podcasts=podcasts.drop(non_eng_pod).drop_duplicates("Name", keep="first")

movies=pd.read_csv("young-people-survey/movie_metadata.csv")
movies=movies[movies["imdb_score"].astype(float)>float(5.0)]
movies=movies.dropna().drop_duplicates("movie_imdb_link", keep="first")

music=pd.read_csv("young-people-survey/SpotifyFeatures.csv")
music=music.drop_duplicates("track_id")
music=music[music["popularity"].astype(float)>float(50)]

genre_IDs=[['1311', 'News & Politics'], ['26', 'Podcasts'], ['1479', 'Social Sciences'], ['1315', 'Science & Medicine'], ['1324', 'Society & Culture'], ['1302', 'Personal Journals'], ['1469', 'Language Courses'], ['1304', 'Education'], ['1320', 'Places & Travel'], ['1416', 'Higher Education'], ['1465', 'Professional'], ['1316', 'Sports & Recreation'], ['1303', 'Comedy'], ['1305', 'Kids & Family'], ['1439', 'Christianity'], ['1314', 'Religion & Spirituality'], ['1444', 'Spirituality'], ['1309', 'TV & Film'], ['1462', 'History'], ['1310', 'Music'], ['1478', 'Medicine'], ['1321', 'Business'], ['1412', 'Investing'], ['1420', 'Self-Help'], ['1307', 'Health'], ['1481', 'Alternative Health'], ['1417', 'Fitness & Nutrition'], ['1467', 'Amateur'], ['1480', 'Software How-To'], ['1318', 'Technology'], ['1448', 'Tech News'], ['1456', 'Outdoor'], ['1477', 'Natural Sciences'], ['1301', 'Arts'], ['1454', 'Automotive'], ['1323', 'Games & Hobbies'], ['1438', 'Buddhism'], ['1443', 'Philosophy'], ['1401', 'Literature'], ['1402', 'Design'], ['1410', 'Careers'], ['1470', 'Training'], ['1413', 'Management & Marketing'], ['1306', 'Food'], ['1406', 'Visual Arts'], ['1446', 'Gadgets'], ['1468', 'Educational Technology'], ['1405', 'Performing Arts'], ['1460', 'Hobbies'], ['1471', 'Business News'], ['1404', 'Video Games'], ['1450', 'Podcasting'], ['1473', 'National'], ['1325', 'Government & Organizations'], ['1461', 'Other Games'], ['1466', 'College & High School'], ['1459', 'Fashion & Beauty'], ['1476', 'Non-Profit'], ['1415', 'K-12'], ['1455', 'Aviation'], ['1464', 'Other'], ['1421', 'Sexuality'], ['1472', 'Shopping'], ['1475', 'Local'], ['1441', 'Judaism'], ['1440', 'Islam'], ['1474', 'Regional'], ['1463', 'Hinduism']]

In [167]:
movies.genres

0           Action|Adventure|Fantasy|Sci-Fi
1                  Action|Adventure|Fantasy
2                 Action|Adventure|Thriller
3                           Action|Thriller
5                   Action|Adventure|Sci-Fi
                       ...                 
5026                    Drama|Music|Romance
5027                                  Drama
5033                  Drama|Sci-Fi|Thriller
5035    Action|Crime|Drama|Romance|Thriller
5042                            Documentary
Name: genres, Length: 3336, dtype: object

In [4]:
# Better organize the id vs name of genre
dict_pod={int(id[0]):id[1] for id in genre_IDs }

# {Genre ID:NAMES OF PODCAST}
genre_to_podcast={key:[] for key in dict_pod }
for item in podcasts.index:
    array=podcasts["Genre IDs"][item][1:-1].split(',')
    array=[int(array[i].replace('\'', '').strip('\'')) for i in range(len(array))]
    for j in range(len(array)):
        genre_to_podcast[array[j]].append((podcasts["Name"][item], podcasts["Description"][item], podcasts["Podcast URL"][item]))
final_dict={list(dict_pod.values())[p]:list(genre_to_podcast.values())[p] for p in range(len(genre_to_podcast.keys()))}
final_dict = dict((k.lower(), v) for k, v in final_dict.items())

#MAKE MOVIE DICTIONARY
genre_to_movie={}
for i in range( len(movies)):
    line=str(movies.iloc[i]["Genre"])
    for genre in line.split("|"):
        genre=genre.lower()
        if genre in genre_to_movie:
            genre_to_movie[genre].append(movies.iloc[i]["Title"])
        else:
            genre_to_movie[genre]=[movies.iloc[i]["Title"]]

#MAKE MUSIC DICTIONARY
genre_to_music={}
for i in range(len(music)):
    genre=music.iloc[i]['genre']
    genre=genre.lower()
    if genre in genre_to_music:
            genre_to_music[genre].append((music.iloc[i]['track_id'], music.iloc[i]['track_name'],  music.iloc[i]['genre'],  music.iloc[i]['artist_name']))
    else:
            genre_to_music[genre]=[(music.iloc[i]['track_id'], music.iloc[i]['track_name'],  music.iloc[i]['genre'],  music.iloc[i]['artist_name'])]


KeyError: 'Genre'

In [5]:
song=recs(genre_to_music, ["Folk"])

NameError: name 'recs' is not defined

In [ ]:
song=[{
            'title': item[0][1],
            'url': "https://open.spotify.com/embed/track/"+item[0][0],
            'artist': item[0][3], 
            'genre': item[0][2]
            } for item in song]

In [ ]:
podcast=recs(final_dict,["News & Politics"])

In [ ]:
podcast=[{
            'title': item[0][0],
            'description': item[0][1],
            'url': item[0][2]
            } for item in podcast]

In [7]:
def cosine_sim(corpus):
    vectorizer=TfidfVectorizer(stop_words="english", min_df=1)
    tfidf = vectorizer.fit_transform(corpus)
    similarity = tfidf * tfidf.T
    return similarity.toarray()

In [8]:
cosine_sim(["I'd like an apple",  "An apple a day keeps the doctor away"])

array([[1.        , 0.19431434],
       [0.19431434, 1.        ]])

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,...,738.0,English,USA,PG-13,263700000.0,2012.0,632.0,6.6,2.35,24000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5026,Color,Olivier Assayas,81.0,110.0,107.0,45.0,Béatrice Dalle,576.0,136007.0,Drama|Music|Romance,...,39.0,French,France,R,4500.0,2004.0,133.0,6.9,2.35,171
5027,Color,Jafar Panahi,64.0,90.0,397.0,0.0,Nargess Mamizadeh,5.0,673780.0,Drama,...,26.0,Persian,Iran,Not Rated,10000.0,2000.0,0.0,7.5,1.85,697
5033,Color,Shane Carruth,143.0,77.0,291.0,8.0,David Sullivan,291.0,424760.0,Drama|Sci-Fi|Thriller,...,371.0,English,USA,PG-13,7000.0,2004.0,45.0,7.0,1.85,19000
5035,Color,Robert Rodriguez,56.0,81.0,0.0,6.0,Peter Marquardt,121.0,2040920.0,Action|Crime|Drama|Romance|Thriller,...,130.0,Spanish,USA,R,7000.0,1992.0,20.0,6.9,1.37,0


In [125]:
def get_max_val_podcast(np_array): 
    index_max_val=np.argmax(np_array)
    output=(podcasts.iloc[index_max_val][:]["Name"], podcasts.iloc[index_max_val][:]["Description"], podcasts.iloc[index_max_val][:]["Podcast URL"], np_array[index_max_val])
    if np_array[index_max_val]==0:
        return 
    else: 
        np_array[index_max_val]=0
    return output

def get_max_val_movie(np_array):
    index_max_val=np.argmax(np_array)
    output=(movies.iloc[index_max_val][:]["movie_title"], movies.iloc[index_max_val][:]["title_year"], movies.iloc[index_max_val][:]["num_voted_users"], movies.iloc[index_max_val][:]["movie_imdb_link"], movies.iloc[index_max_val][:]["imdb_score"],np_array[index_max_val])
    if np_array[index_max_val]==0:
        return 
    else: 
        np_array[index_max_val]=0
    return output

def podcast_recs(query):
    descriptions=list(podcasts['Description'])
    corpus=[query]+descriptions
    matrix=cosine_sim(corpus)
    matrix_slice=matrix[:][0][1:]
    result=[]
    for x in range(5):
        result.append(get_max_val_podcast(matrix_slice))
    while None in result:
        result.remove(None)
    return result

def movie_recs(query, genres=None):
    if genres is not None:
        movies_filtered = movies
    else:
        movies_filtered = movies[movies.apply(lambda x: x['genre'] in genres, axis=1)]
        
    plot_keywords=list(movies_filtered['plot_keywords'])
    for i in range(len(plot_keywords)):
        plot_keywords[i]=plot_keywords[i].replace('|', " ")
    corpus=[query]+plot_keywords
    matrix=cosine_sim(corpus)
    matrix_slice=matrix[:][0][1:]
    result=[]
    for x in range(5):
        movie=get_max_val_movie(matrix_slice)
        result.append(movie)
    while None in result:
        result.remove(None)
    return result

In [126]:
movie_recs("cooking")

0
1
2
3
4


[('Julie & Julia\xa0',
  2009.0,
  79264,
  'http://www.imdb.com/title/tt1135503/?ref_=fn_tt_tt_1',
  7.0,
  0.5069031330115615),
 ('No Reservations\xa0',
  2007.0,
  59068,
  'http://www.imdb.com/title/tt0481141/?ref_=fn_tt_tt_1',
  6.3,
  0.47573072030813696)]

In [112]:
movie

[('No Reservations\xa0',
  2007.0,
  59068,
  'http://www.imdb.com/title/tt0481141/?ref_=fn_tt_tt_1',
  6.3,
  0.42245193993383945),
 ('Ghost\xa0',
  1990.0,
  154199,
  'http://www.imdb.com/title/tt0099653/?ref_=fn_tt_tt_1',
  7.0,
  0.21355121535489127),
 ('The Brothers McMullen\xa0',
  1995.0,
  6375,
  'http://www.imdb.com/title/tt0112585/?ref_=fn_tt_tt_1',
  6.6,
  0.1897134201800415),
 ('Practical Magic\xa0',
  1998.0,
  55749,
  'http://www.imdb.com/title/tt0120791/?ref_=fn_tt_tt_1',
  6.1,
  0.15523674319992337),
 ('My Life Without Me\xa0',
  2003.0,
  21202,
  'http://www.imdb.com/title/tt0314412/?ref_=fn_tt_tt_1',
  7.6,
  0.15125469060407867)]

In [93]:
movie=[{
            'title': item[0],
            'year': item[1],
            'link': item[3], 
            'num_revs': item[2],
            'rating': item[4],
            'score': item[5]
            } for item in movie]

TypeError: 'NoneType' object is not subscriptable

In [12]:
movie

[{'link': 'http://www.imdb.com/title/tt1127180/?ref_=fn_tt_tt_1',
  'num_revs': 158354,
  'rating': 6.6,
  'score': 0.41124307355079887,
  'title': 'Drag Me to Hell\xa0',
  'year': 2009.0},
 {'link': 'http://www.imdb.com/title/tt0790686/?ref_=fn_tt_tt_1',
  'num_revs': 86205,
  'rating': 6.2,
  'score': 0.38687880213989567,
  'title': 'Mirrors\xa0',
  'year': 2008.0},
 {'link': 'http://www.imdb.com/title/tt0450385/?ref_=fn_tt_tt_1',
  'num_revs': 209396,
  'rating': 6.8,
  'score': 0.35623519871145776,
  'title': '1408\xa0',
  'year': 2007.0},
 {'link': 'http://www.imdb.com/title/tt3691740/?ref_=fn_tt_tt_1',
  'num_revs': 12572,
  'rating': 6.8,
  'score': 0.34498343594039377,
  'title': 'The BFG\xa0',
  'year': 2016.0},
 {'link': 'http://www.imdb.com/title/tt3300542/?ref_=fn_tt_tt_1',
  'num_revs': 69484,
  'rating': 5.9,
  'score': 0.3426556446389445,
  'title': 'London Has Fallen\xa0',
  'year': 2016.0}]

In [13]:
for podcast in podcasts:
    print(podcast)

Name
Artwork
Genre IDs
Episode Count
Episode Durations
iTunes URL
Feed URL
Podcast URL
Description


In [127]:
podcast=[{
            'title': item[0],
            'description': item[1],
            'url': item[2], 
            'score': item[3]
            } for item in podcast]

IndexError: tuple index out of range

In [16]:
podcast=podcast_recs("alsfkn")

In [17]:
podcast[0]==()

True

In [18]:
def recs(genre_dict, genre_query):
    """Returns a list of recommendations based on interests user clicked in form
    Params: {query: list of genre names, genre_dict: dictionary that maps genre to titles, corr_dict: maps personality questions to answers given in survey}
    Returns: list of tuples containing titles and scores
    """

    genre_query=[genre.lower() for genre in genre_query]

    counter={}
    for cat in genre_query:
        for film in genre_dict[cat]:
            if film in counter:
                counter[film]+=1
            else:
                counter[film]=1
    results = list(counter.items())
    random.shuffle(results)  
    results.sort(key=lambda x: x[1], reverse=True)

    return results